In [1]:
import emoji

In [2]:

emoji_dictionary={
    "0":":beating_heart:",
    "1":":baseball:",
    "2":":beaming_face_with_smiling_eyes:",
    "3":":downcast_face_with_sweat:",
    "4":":fork_and_knife:"
}

In [3]:

import pandas as pd
import numpy as np

In [5]:
train=pd.read_csv("train_emoji.csv")
test=pd.read_csv("test_emoji.csv")

In [6]:
train.head(n=10)

,never talk to me again,3,Unnamed: 2,Unnamed: 3
0,I am proud of your achievements,2,NaN,NaN
1,It is the worst day in my life,3,NaN,NaN
2,Miss you so much,0,NaN,[0]
3,food is life,4,NaN,NaN
4,I love you mum,0,NaN,NaN
5,Stop saying bullshit,3,NaN,NaN
6,congratulations on your acceptance,2,NaN,NaN
7,The assignment is too long,3,NaN,NaN
8,I want to go play,1,NaN,[3]
9,she did not answer my text,3,NaN,NaN


In [7]:
train=train.values

In [8]:
x_train=train[:,0]
y_train=train[:,1]

In [9]:
print(x_train.shape,y_train.shape)

(131,) (131,)


In [10]:

test=test.values
x_test=test[:,0]
y_test=test[:,1]
print(x_test.shape,y_test.shape)

(55,) (55,)


In [11]:
for i in range(5):
    print(x_train[i],emoji.emojize(emoji_dictionary.get(str(y_train[i]))))

I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much 💓
food is life 🍴
I love you mum 💓


In [12]:
from keras.utils import to_categorical

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [13]:

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
print(y_train.shape,y_test.shape)

(131, 5) (55, 5)


In [14]:
f=open("glove.6B.50d.txt",encoding="utf8")

In [15]:

embeddings={}
for line in f:
    data=line.split()
    word=data[0]
    values=np.array(data[1:],dtype="float32")
    embeddings[word]=values
f.close()

In [16]:
print(embeddings["hello"])

[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
 -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
  0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
  0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
  0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
 -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
 -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
  0.67204 ]


In [17]:

print(embeddings["shape"].shape)

(50,)


In [18]:

def embedding_output(X,maxlen=10):
    embedding_output=np.zeros((X.shape[0],maxlen,50))
    for i in range(X.shape[0]):
        X[i]=X[i].split()
        for j in range(len(X[i])):
            try:
                embedding_output[i][j]=embeddings[X[i][j].lower()]
            except:
                embedding_output[i][j]=np.zeros((50,))
    return embedding_output

In [19]:

embedded_x_train=embedding_output(x_train)
embedded_x_test=embedding_output(x_test)

In [22]:
print(x_train[0])

['I', 'am', 'proud', 'of', 'your', 'achievements']


In [20]:
print(embedded_x_train.shape,embedded_x_test.shape)

(131, 10, 50) (55, 10, 50)


In [24]:

from keras.models import Sequential
from keras.layers import *

In [47]:

model = Sequential()
model.add(LSTM(128,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation("softmax"))
model.summary()
         
          

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
Total params: 141,381
Trainable params: 141,381
Non-trainable params: 0
________________________________________________

In [48]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [49]:
from keras.callbacks import ModelCheckpoint

Checkpoint=ModelCheckpoint("best_model.h5",monitor='val_acc',save_best_only=True)

In [50]:
hist=model.fit(embedded_x_train,y_train,epochs=80,batch_size=64,shuffle=True,validation_split=0.2,callbacks=[Checkpoint])

Train on 104 samples, validate on 27 samples
Epoch 1/80
104/104 [==============================] - 2s 16ms/step - loss: 1.5627 - accuracy: 0.3269 - val_loss: 1.6170 - val_accuracy: 0.2222
Epoch 2/80
104/104 [==============================] - 0s 1ms/step - loss: 1.5238 - accuracy: 0.3269 - val_loss: 1.6439 - val_accuracy: 0.2222
Epoch 3/80
 64/104 [=================>............] - ETA: 0s - loss: 1.4480 - accuracy: 0.3906

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


104/104 [==============================] - 0s 930us/step - loss: 1.4929 - accuracy: 0.3654 - val_loss: 1.6776 - val_accuracy: 0.2593
Epoch 4/80
104/104 [==============================] - 0s 825us/step - loss: 1.4486 - accuracy: 0.4423 - val_loss: 1.6778 - val_accuracy: 0.2222
Epoch 5/80
104/104 [==============================] - 0s 853us/step - loss: 1.4254 - accuracy: 0.4038 - val_loss: 1.6537 - val_accuracy: 0.2222
Epoch 6/80
104/104 [==============================] - 0s 1ms/step - loss: 1.3994 - accuracy: 0.4519 - val_loss: 1.6065 - val_accuracy: 0.2222
Epoch 7/80
104/104 [==============================] - 0s 1000us/step - loss: 1.3470 - accuracy: 0.4904 - val_loss: 1.5654 - val_accuracy: 0.2222
Epoch 8/80
104/104 [==============================] - 0s 1ms/step - loss: 1.2696 - accuracy: 0.5192 - val_loss: 1.5132 - val_accuracy: 0.2963
Epoch 9/80
104/104 [==============================] - 0s 795us/step - loss: 1.1738 - accuracy: 0.5962 - val_loss: 1.4563 - val_accuracy: 0.3333
Epoch 

Epoch 60/80
104/104 [==============================] - 0s 950us/step - loss: 0.0441 - accuracy: 0.9904 - val_loss: 1.7026 - val_accuracy: 0.6296
Epoch 61/80
104/104 [==============================] - 0s 831us/step - loss: 0.0227 - accuracy: 1.0000 - val_loss: 1.8140 - val_accuracy: 0.5926
Epoch 62/80
104/104 [==============================] - 0s 748us/step - loss: 0.0291 - accuracy: 0.9904 - val_loss: 1.7753 - val_accuracy: 0.6296
Epoch 63/80
104/104 [==============================] - 0s 835us/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 1.9161 - val_accuracy: 0.6667
Epoch 64/80
104/104 [==============================] - 0s 1ms/step - loss: 0.0649 - accuracy: 0.9712 - val_loss: 1.6111 - val_accuracy: 0.6296
Epoch 65/80
104/104 [==============================] - 0s 902us/step - loss: 0.0221 - accuracy: 1.0000 - val_loss: 1.2525 - val_accuracy: 0.6296
Epoch 66/80
104/104 [==============================] - 0s 929us/step - loss: 0.0351 - accuracy: 0.9904 - val_loss: 1.4274 - val_accu

In [55]:

model.evaluate(embedded_x_test,y_test)

55/55 [==============================] - 0s 329us/step


[2.0276828397404065, 0.6363636255264282]

In [56]:
predict=model.predict_classes(embedded_x_test)

In [57]:

for i in range(10):
    print(" ".join(x_test[i]),emoji.emojize(emoji_dictionary[str(predict[i])]))

he did not answer 😓
he got a raise 😁
she got me a present 😁
ha ha ha it was so funny 😁
he is a good friend 😁
I am upset ⚾
We had such a lovely dinner tonight 😁
where is the food 🍴
Stop making this joke ha ha ha 😁
where is the ball ⚾
